In [0]:
from keras.datasets import cifar10

Using TensorFlow backend.


In [0]:
(Train_Data, Train_Labels), (Test_Data, Test_Labels) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
Train_Data.shape

(50000, 32, 32, 3)

In [0]:
from keras.utils import to_categorical
Train_Labels = to_categorical(Train_Labels)
Test_Labels = to_categorical(Test_Labels)

#Train_Data = Train_Data.astype('float32') / 255
#Test_Data = Test_Data.astype('float32') / 255

In [0]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(32, 32, 3))

In [0]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [0]:
from keras import layers
from keras import models
from keras import regularizers
from keras import optimizers

model = models.Sequential()
model.add(conv_base)
#model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.005)))
model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 14,748,170
Trainable params: 14,748,170
Non-trainable params: 0
_________________________________________________________________


In [0]:
#  Freezing the extracted convolutional base
#  conv_base.trainable = False

# Fine Tuning
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False
    
len(model.trainable_weights)

10

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
from keras import applications

train_datagen = ImageDataGenerator(
    preprocessing_function = applications.vgg16.preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_datagen.fit(Train_Data)

In [0]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [0]:
#model.fit(Train_Data, Train_Labels, batch_size=64, epochs=15)
model.fit_generator(train_datagen.flow(Train_Data, Train_Labels, batch_size=64),\
                    steps_per_epoch=len(Train_Data) /64, epochs=30)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
782/781 [==============================] - 32s 41ms/step - loss: 14.5129 - acc: 0.1017
Epoch 2/30
 25/781 [..............................] - ETA: 21s - loss: 14.5467 - acc: 0.0975

KeyboardInterrupt: ignored

In [0]:
test_datagen = ImageDataGenerator(preprocessing_function= applications.vgg16.preprocess_input)

test_loss, test_acc = model.evaluate_generator(test_datagen.flow(Test_Data, Test_Labels, batch_size=64), steps=len(Test_Data) /64)

In [0]:
print("Test Accuracy:",test_acc)
print("Test Loss:",test_loss)